In [1]:
import numpy as np
import socket
import threading
import time
import pickle
import random
import matplotlib.pyplot as plt
from networkx import nx


In [2]:
class process(threading.Thread):
    
 
    def __init__(self,idd,r1,r2,s):
        
        
        
        threading.Thread.__init__(self)
        
        
        
        self.id=idd
        self.r1=r1
        self.r2=r2
        self.state=s
        self.ack=0
        self.ps=0
        self.ss=0
        self.port=4000+self.id
        
    
    
    def listen(self):
        
        
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.bind(('127.0.0.1',self.port))
        sock.listen(10)
        
        
            
            
        while True:
            
            
            
            connection, client_address = sock.accept()
                
            while True:
                      
                
                
                data1 = connection.recv(1024);
                
                
                if data1 :
                          
                        
                    kc=pickle.loads(data1)
                  
                    if kc['type']=='port_to_send':
                  
                        self.ps=kc['value']
                        self.ss='send1'
                  
                  
                    if kc['type']=='send1':
                        
                  
                  
                  
                        self.mess=kc['value'][0]
                        self.ps=kc['value'][1]
                        
                        kk=[(self.r1.index(item),item[0]) for item in self.r1 if (item[0] == self.mess and item[1]==self.state)or (item[0] == self.state and item[1]==self.mess)]
                  
                        
                  
                        if not kk:
                            self.state=self.state
                            self.mess=self.mess
                  
                        elif kk[0][1]==self.state:
                            self.state=self.r2[kk[0][0]][0]
                            self.mess=self.r2[kk[0][0]][1]
                            
                        elif kk[0][1]==self.mess:
                  
                            self.state=self.r2[kk[0][0]][1]
                            self.mess=self.r2[kk[0][0]][0]
                  
                        self.ack='ack1'
                        self.ss='send2'
                  
                  
                    if kc['type']=='send2':
                        self.state=kc['value']
                        
                        self.ack='ack2'
                  
                  
                  
                  
                    
                    
                    
                    
   
                if not data1:
                      
                    break;
                  
                      
                      
                
                        
        return                
                    

    def send(self):
        
        
        
    
        while True:
            
                
            
            if self.ss=='send1':
                      
                
                
                    
                self.ss=0
                
                data={};
                
                data['type']="send1"
                data['value']=self.state,self.port
                x=pickle.dumps(data)
                s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                s.connect(('127.0.0.1',self.ps))
                s.sendall(x)    
                s.close()
        
        
                
            if self.ss=='send2':
                      
                      
                
                
                    
                self.ss=0
                
                data={};
                
                data['type']="send2"
                data['value']=self.mess
                x=pickle.dumps(data)
                s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                s.connect(('127.0.0.1',self.ps))
                s.sendall(x)    
                s.close()
                
            if self.ack=='ack1' or self.ack=='ack2':
                      
                
                
                    
                
                
                data={};
                
                data['type']=self.ack
                self.ack=0
                data['value']=self.id,self.state
                x=pickle.dumps(data)
                s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                s.connect(('127.0.0.1',9090))
                s.sendall(x)    
                s.close()    
    
        return
    
    

    def run1(self):
                  
        
        t=[]
        
        
        
        t1=threading.Thread(target=self.listen)
        t2=threading.Thread(target=self.send)
        t2.start()        
        t1.start()
        t.append(t1)
        t.append(t2)
        for n in t:
                  
            
            n.join()
         
    

In [3]:
#complete graph
class sch1(threading.Thread):
 
    def __init__(self,n,s,o,cs,co):
        
        
        #s states
        #o outputs
        #target output
        #cs random states
        #co random outputs
        
        
        
        threading.Thread.__init__(self)
        self.t1=time.time()
        self.size=n
        self.ack=0
        self.start=1
        
        self.s=s
        
        self.o=o
        
        self.message=0
        self.end=0
        
        self.configs=cs
        self.configo=co
        
        self.ids=list(range(self.size))
        self.ports=[x+4000 for x in self.ids]
        
        
        
        
        
        
    def send(self):
        
        while True:
            
            if self.start==1 or self.ack==2:
                
                self.start=0
                self.ack=0
                self.message=self.message+1
                c=random.sample(self.ids,2)
                
                data={};
                data['type']="port_to_send"
                data['value']=self.ports[c[1]]
                x=pickle.dumps(data)           
                
                
                s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                s.connect(('127.0.0.1',self.ports[c[0]]))
                s.sendall(x)    
                s.close()
        
            
            if self.end== 1:
                
                
                
                break;
            
            
            
    def listen(self):
        
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.bind(('127.0.0.1',9090))
        sock.listen(10)
           
        while True:
            
            
            
            connection, client_address = sock.accept()
                
            while True:
                
                
                data1 = connection.recv(1024);
                
                
                if data1 :
                        
                    kc=pickle.loads(data1)
                  
                  
                    if kc['type']=='ack1':
                  
                  
                  
                  
                        self.configs[kc['value'][0]]=kc['value'][1]
                  
                        self.message=self.message+2
                        self.ack=self.ack+1
                        self.configo[kc['value'][0]]=self.o[self.s.index(kc['value'][1])]
                  
                    
                    if kc['type']=='ack2':
                  
                  
                        self.configs[kc['value'][0]]=kc['value'][1]
                        self.message=self.message+2
                        self.configo[kc['value'][0]]=self.o[self.s.index(kc['value'][1])]
                        self.ack=self.ack+1
                  
                  
                    if len(set(self.configo))== 1:
                        self.end=1
                        print('Time:'+str(time.time()-self.t1)+'   message:'+str(self.message))
                        
                        break;
                    
                    
                if not data1:
                  
                  break;
        
        
                 
                        
        return     
        
    def run1(self):
                  
        
        t=[]
        
        
        
        t1=threading.Thread(target=self.listen)
        t2=threading.Thread(target=self.send)
        t2.start()        
        t1.start()
        t.append(t1)
        t.append(t2)
        for n in t:
                  
            
            n.join() 

In [4]:
#grid graph without mobility
class sch2(threading.Thread):
 
    def __init__(self,q1,q2,s,o,cs,co):
        
        #r1 first part of transition rule
        #r2 second part of transition rule
        #s states
        #o outputs
        #q1 tool
        #q2 arz
        #cs random states
        #co random outputs
        
        
        
        threading.Thread.__init__(self)
        self.t1=time.time()
        self.q1=q1
        self.q2=q2
        self.size=q1*q2
        self.ack=0
        self.start=1
        
        self.s=s
        self.o=o
        
        self.message=0
        
        
        self.configs=cs
        self.configo=co
        
        
        
        
        self.ids=list(range(self.size))
        self.ports=[x+4000 for x in self.ids]
        
        
        
        
    def send(self):
        
        while True:
            
            if self.start==1 or self.ack==2:
                
                self.start=0
                self.ack=0
                self.message=self.message+1
                
                c=random.sample(self.ids,1)[0]
                
                x=c%self.q1
                y=c//self.q1
                
                ney=[]
                
                if x+1<self.q1:
                    ney.append((x+1,y))
                if x-1>=0:
                    ney.append((x-1,y))
                if y+1<self.q2:
                    ney.append((x,y+1))
                if y-1>=0:
                    ney.append((x,y-1))
                    
                w=random.sample(ney,1)[0]
                
                

                data={};
                data['type']="port_to_send"
                data['value']=self.ports[w[0]+w[1]*self.q1]
                x=pickle.dumps(data)           
                
                
                s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                s.connect(('127.0.0.1',self.ports[c]))
                s.sendall(x)    
                s.close()
        
            
            if len(set(self.configo)) == 1:
                
                print('Time:'+str(time.time()-self.t1)+'   message:'+str(self.message))
                break;
            
            
            
    def listen(self):
        
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.bind(('127.0.0.1',9090))
        sock.listen(10)
           
        while True:
            
            
            
            connection, client_address = sock.accept()
                
            while True:
                
                
                data1 = connection.recv(1024);
                
                
                if data1 :
                        
                    kc=pickle.loads(data1)
                  
                  
                    if kc['type']=='ack1':
                  
                  
                  
                  
                        self.configs[kc['value'][0]]=kc['value'][1]
                  
                        self.message=self.message+2
                        self.ack=self.ack+1
                  
                        self.configo[kc['value'][0]]=self.o[self.s.index(kc['value'][1])]
                  
                    
                    if kc['type']=='ack2':
                  
                  
                        self.configs[kc['value'][0]]=kc['value'][1]
                        self.message=self.message+2
                        self.configo[kc['value'][0]]=self.o[self.s.index(kc['value'][1])]
                        self.ack=self.ack+1
                  
                  
   
                if not data1:
                  
                  break;
        
               
        return     
        
    def run1(self):
                  
        
        t=[]
        
        
        
        t1=threading.Thread(target=self.listen)
        t2=threading.Thread(target=self.send)
        t2.start()        
        t1.start()
        t.append(t1)
        t.append(t2)
        for n in t:
                  
            
            n.join()   

In [5]:
#mesh graph without mobility
class sch3(threading.Thread):
 
    def __init__(self,q1,q2,s,o,cs,co):
        
        #r1 first part of transition rule
        #r2 second part of transition rule
        #s states
        #o outputs
        #q1 tool
        #q2 arz
        #cs random states
        #co random outputs
        
        
        
        threading.Thread.__init__(self)
        self.t1=time.time()
        self.q1=q1
        self.q2=q2
        self.size=q1*q2
        self.ack=0
        self.start=1
        
        self.s=s
        self.o=o
        
        self.message=0
        
        
        self.configs=cs
        self.configo=co
        
        
        
        self.ids=list(range(self.size))
        self.ports=[x+4000 for x in self.ids]
        
        
        
        
    def send(self):
        
        while True:
            
            if self.start==1 or self.ack==2:
                
                self.start=0
                self.ack=0
                self.message=self.message+1
                
                c=random.sample(self.ids,1)[0]
                
                x=c%self.q1
                y=c//self.q1
                
                ney=[]
                
                if x+1<self.q1:
                    ney.append((x+1,y))
                if x-1>=0:
                    ney.append((x-1,y))
                if y+1<self.q2:
                    ney.append((x,y+1))
                if y-1>=0:
                    ney.append((x,y-1))
                if x+1==self.q1:
                    ney.append((x-(self.q1-1),y))
                if x-1==-1:
                    ney.append((x+(self.q1-1),y))
                if y+1==self.q2:
                    ney.append((x,y-(self.q2-1)))
                if y-1==-1:
                    ney.append((x,y+(self.q2-1)))
                    
                w=random.sample(ney,1)[0]
                
                

                data={};
                data['type']="port_to_send"
                data['value']=self.ports[w[0]+w[1]*self.q1]
                x=pickle.dumps(data)           
                
                
                s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                s.connect(('127.0.0.1',self.ports[c]))
                s.sendall(x)    
                s.close()
        
            
            if len(set(self.configo)) == 1:
                
                print('Time:'+str(time.time()-self.t1)+'   message:'+str(self.message))
                break;
            
            
            
    def listen(self):
        
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.bind(('127.0.0.1',9090))
        sock.listen(10)
           
        while True:
            
            
            
            connection, client_address = sock.accept()
                
            while True:
                
                
                data1 = connection.recv(1024);
                
                
                if data1 :
                        
                    kc=pickle.loads(data1)
                  
                  
                    if kc['type']=='ack1':
                  
                  
                  
                  
                        self.configs[kc['value'][0]]=kc['value'][1]
                  
                        self.message=self.message+2
                        self.ack=self.ack+1
                  
                        self.configo[kc['value'][0]]=self.o[self.s.index(kc['value'][1])]
                  
                    
                    if kc['type']=='ack2':
                  
                  
                        self.configs[kc['value'][0]]=kc['value'][1]
                        self.message=self.message+2
                        self.configo[kc['value'][0]]=self.o[self.s.index(kc['value'][1])]
                        self.ack=self.ack+1
                  
                  
   
                if not data1:
                  
                  break;
        
               
        return     
        
    def run1(self):
                  
        
        t=[]
        
        
        
        t1=threading.Thread(target=self.listen)
        t2=threading.Thread(target=self.send)
        t2.start()        
        t1.start()
        t.append(t1)
        t.append(t2)
        for n in t:
                  
            
            n.join()   

In [6]:
#mesh graph with mobility
class sch4(threading.Thread):
 
    def __init__(self,q1,q2,s,o,cs,co):
        
        #r1 first part of transition rule
        #r2 second part of transition rule
        #s states
        #o outputs
        #q1 tool
        #q2 arz
        #cs random states
        #co random outputs
        
        
        
        threading.Thread.__init__(self)
        self.t1=time.time()
        self.q1=q1
        self.q2=q2
        self.size=q1*q2
        self.ack=0
        self.start=1
        
        self.s=s
        self.o=o
        
        self.message=0
        
        
        self.configs=cs
        self.configo=co
        
        
        
        self.ids=list(range(self.size))
        self.squre=self.ids
        self.direction=[]
        self.ports=[]
        for e in range(self.size):
            self.ports.append(e+4000)
            
            if random.uniform(0,1)<0.5:
                
                self.direction.append('vertical')
            else:
                self.direction.append('horizental')
        
        
        
        
    def send(self):
        
        while True:
            
            if self.start==1 or self.ack==2:
                
                self.start=0
                self.ack=0
                self.message=self.message+1
                
                for u in range(self.size):
                    if self.direction[u]=='vertical':
                        x=self.squre[u]%self.q1
                        y=random.randint(0,self.q2-1)
                        self.squre[u]=x+self.q1*y
                        
                    elif self.direction[u]=='horizental':
                        
                        x=random.randint(0,self.q1-1)
                        y=self.squre[u]//self.q1
                        self.squre[u]=x+self.q1*y
                    
                    
                
                while True:
                    
                    c=random.randint(0,self.size-1)
                    c1=self.squre[c]
                    x=c1%self.q1
                    y=c1//self.q1
                
                
                    ney=[(x,y)]
                
                    if x+1<self.q1:
                        ney.append((x+1,y))
                    if x-1>=0:
                        ney.append((x-1,y))
                    if y+1<self.q2:
                        ney.append((x,y+1))
                    if y-1>=0:
                        ney.append((x,y-1))
                    if x+1==self.q1:
                        ney.append((x-(self.q1-1),y))
                    if x-1==-1:
                        ney.append((x+(self.q1-1),y))
                    if y+1==self.q2:
                        ney.append((x,y-(self.q2-1)))
                    if y-1==-1:
                        ney.append((x,y+(self.q2-1)))
                    
                    
                   
                        
                    w=random.sample(ney,1)[0]
                    w1=[i for i,value in enumerate(self.squre) if value == w[0]+w[1]*self.q1]
                       
                    
                    
                    if w1:
                        w2=random.sample(w1,1)[0]
                        if w2!=c:
                            
                            break
                
                

                data={};
                data['type']="port_to_send"
                data['value']=self.ports[w2]
                x=pickle.dumps(data)           
                
                
                s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                s.connect(('127.0.0.1',self.ports[c]))
                s.sendall(x)    
                s.close()
        
            
            if len(set(self.configo)) == 1:
                
                print('Time:'+str(time.time()-self.t1)+'   message:'+str(self.message))
                break;
            
            
            
    def listen(self):
        
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.bind(('127.0.0.1',9090))
        sock.listen(10)
           
        while True:
            
            
            
            connection, client_address = sock.accept()
                
            while True:
                
                
                data1 = connection.recv(1024);
                
                
                if data1 :
                        
                    kc=pickle.loads(data1)
                  
                  
                    if kc['type']=='ack1':
                  
                  
                  
                  
                        self.configs[kc['value'][0]]=kc['value'][1]
                  
                        self.message=self.message+2
                        self.ack=self.ack+1
                  
                        self.configo[kc['value'][0]]=self.o[self.s.index(kc['value'][1])]
                  
                    
                    if kc['type']=='ack2':
                  
                  
                        self.configs[kc['value'][0]]=kc['value'][1]
                        self.message=self.message+2
                        self.configo[kc['value'][0]]=self.o[self.s.index(kc['value'][1])]
                        self.ack=self.ack+1
                  
                  
   
                if not data1:
                  
                  break;
        
               
        return     
        
    def run1(self):
                      
        
        t=[]
        
        
        
        t1=threading.Thread(target=self.listen)
        t2=threading.Thread(target=self.send)
        t2.start()        
        t1.start()
        t.append(t1)
        t.append(t2)
        for n in t:
                  
            
            n.join()   

In [7]:
#grid graph with mobility
class sch5(threading.Thread):
 
    def __init__(self,q1,q2,s,o,cs,co):
        
        #r1 first part of transition rule
        #r2 second part of transition rule
        #s states
        #o outputs
        #q1 tool
        #q2 arz
        #cs random states
        #co random outputs
        
        
        
        threading.Thread.__init__(self)
        self.t1=time.time()
        self.q1=q1
        self.q2=q2
        self.size=q1*q2
        self.ack=0
        self.start=1
        
        self.s=s
        self.o=o
        
        self.message=0
        
        
        self.configs=cs
        self.configo=co
        
        
        
        self.ids=list(range(self.size))
        self.squre=self.ids
        self.direction=[]
        self.ports=[]
        for e in range(self.size):
            self.ports.append(e+4000)
            
            if random.uniform(0,1)<0.5:
                
                self.direction.append('vertical')
            else:
                self.direction.append('horizental')
        
        
        
        
    def send(self):
        
        while True:
            
            if self.start==1 or self.ack==2:
                
                self.start=0
                self.ack=0
                self.message=self.message+1
                
                for u in range(self.size):
                    if self.direction[u]=='vertical':
                        x=self.squre[u]%self.q1
                        y=random.randint(0,self.q2-1)
                        self.squre[u]=x+self.q1*y
                        
                    elif self.direction[u]=='horizental':
                        
                        x=random.randint(0,self.q1-1)
                        y=self.squre[u]//self.q1
                        self.squre[u]=x+self.q1*y
                    
                    
                
                while True:
                    
                    c=random.randint(0,self.size-1)
                    c1=self.squre[c]
                    x=c1%self.q1
                    y=c1//self.q1
                
                
                    ney=[(x,y)]
                
                    if x+1<self.q1:
                        
                        ney.append((x+1,y))
                    if x-1>=0:
                        ney.append((x-1,y))
                    if y+1<self.q2:
                        ney.append((x,y+1))
                    if y-1>=0:
                        ney.append((x,y-1))
                    
                    w=random.sample(ney,1)[0]
                    w1=[i for i,value in enumerate(self.squre) if value == w[0]+w[1]*self.q1]
                    if w1:
                        w2=random.sample(w1,1)[0]
                        if w2!=c:
                            
                            break
                
                

                data={};
                data['type']="port_to_send"
                data['value']=self.ports[w2]
                x=pickle.dumps(data)           
                
                
                s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                s.connect(('127.0.0.1',self.ports[c]))
                s.sendall(x)    
                s.close()
        
            
            if len(set(self.configo)) == 1:
                
                print('Time:'+str(time.time()-self.t1)+'   message:'+str(self.message))
                break;
            
            
            
    def listen(self):
        
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.bind(('127.0.0.1',9090))
        sock.listen(10)
           
        while True:
            
            
            
            connection, client_address = sock.accept()
                
            while True:
                
                
                data1 = connection.recv(1024);
                
                
                if data1 :
                        
                    kc=pickle.loads(data1)
                  
                  
                    if kc['type']=='ack1':
                  
                  
                  
                  
                        self.configs[kc['value'][0]]=kc['value'][1]
                  
                        self.message=self.message+2
                        self.ack=self.ack+1
                  
                        self.configo[kc['value'][0]]=self.o[self.s.index(kc['value'][1])]
                  
                    
                    if kc['type']=='ack2':
                  
                  
                        self.configs[kc['value'][0]]=kc['value'][1]
                        self.message=self.message+2
                        self.configo[kc['value'][0]]=self.o[self.s.index(kc['value'][1])]
                        self.ack=self.ack+1
                  
                  
   
                if not data1:
                  
                  break;
        
               
        return     
        
    def run1(self):
                  
        
        t=[]
        
        
        
        t1=threading.Thread(target=self.listen)
        t2=threading.Thread(target=self.send)
        t2.start()        
        t1.start()
        t.append(t1)
        t.append(t2)
        for n in t:
                  
            
            n.join()   

In [8]:
#erdos-rayni graph 
class sch6(threading.Thread):
 
    def __init__(self,n,m,s,o,cs,co):
        
        #r1 first part of transition rule
        #r2 second part of transition rule
        #s states
        #o outputs
        #n vertices
        #m edges
        #cs random states
        #co random outputs
        
        
        
        threading.Thread.__init__(self)
        self.t1=time.time()
        
        self.m=m
        self.size=n
        self.ack=0
        self.start=1
        
        
        
        self.s=s
        self.o=o
        self.G = nx.gnm_random_graph(self.size,self.m)
        
        self.edges=list(self.G.edges)
        
        self.message=0
        
        
        self.configs=cs
        self.configo=co
        
        
        
        self.ids=list(range(self.size))
        self.ports=[]
        for e in range(self.size):
            
            self.ports.append(e+4000)
            
            
        
        
        
        
    def send(self):
        
        while True:
            
            if self.start==1 or self.ack==2:
                
                self.start=0
                self.ack=0
                self.message=self.message+1
                
                c=random.sample(self.edges,1)[0]
                
                
                

                data={};
                data['type']="port_to_send"
                data['value']=self.ports[c[0]]
                x=pickle.dumps(data)           
                
                
                s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                s.connect(('127.0.0.1',self.ports[c[1]]))
                s.sendall(x)    
                s.close()
        
            
            if len(set(self.configo)) == 1:
                
                print('Time:'+str(time.time()-self.t1)+'   message:'+str(self.message))
                break;
            
            
            
    def listen(self):
        
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.bind(('127.0.0.1',9090))
        sock.listen(10)
           
        while True:
            
            
            
            connection, client_address = sock.accept()
                
            while True:
                
                
                data1 = connection.recv(1024);
                
                
                if data1 :
                        
                    kc=pickle.loads(data1)
                  
                  
                    if kc['type']=='ack1':
                  
                  
                  
                  
                        self.configs[kc['value'][0]]=kc['value'][1]
                  
                        self.message=self.message+2
                        self.ack=self.ack+1
                  
                        self.configo[kc['value'][0]]=self.o[self.s.index(kc['value'][1])]
                  
                    
                    if kc['type']=='ack2':
                  
                  
                        self.configs[kc['value'][0]]=kc['value'][1]
                        self.message=self.message+2
                        self.configo[kc['value'][0]]=self.o[self.s.index(kc['value'][1])]
                        self.ack=self.ack+1
                  
                  
   
                if not data1:
                  
                  break;
        
               
        return     
        
    def run1(self):
                  
        
        t=[]
        
        
        
        t1=threading.Thread(target=self.listen)
        t2=threading.Thread(target=self.send)
        t2.start()        
        t1.start()
        t.append(t1)
        t.append(t2)
        for n in t:
                  
            
            n.join()   

In [9]:
r1=[('1','0')]
r2=[('1','1')]

s=['0','1']
o=[0,1]
n=100

In [10]:
oo=['1','0']
co=[]
cs=['1']
for i in range(n-1):
    cs.append('0')
    
random.shuffle(cs)

for i in range(n):
    if cs[i]=='1':
        
        co.append(1)
    else:
        co.append(0)

In [ ]:
def dia(x):
    threads=[];
    for j in range(n):
        sp=process(j,r1,r2,cs[j]);
        t=threading.Thread(target=sp.run1)
        t.start()
        threads.append(t)
        
    kk=sch1(n,s,o,cs,co)    
    t=threading.Thread(target=kk.run1)
    t.start()
    threads.append(t) 
    
    for thread in threads:
        thread.join()
        
dia(1)       